In [5]:
# 本代码由可视化策略环境自动生成 2021年11月8日 12:07
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 1
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    #context.stock_weights = (1,0)
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 1
    context.options['hold_days'] = 1
    #用于判断奇偶交易
    context.datecont = 0

# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    if context.datecont == 0:
        context.datecont = 1
    else:
        context.datecont = 0
    
    positions = {e.symbol: p for e, p in context.portfolio.positions.items() if p.amount>0}
    
    #大盘风控模块，读取风控数据
    today = data.current_dt.strftime('%Y-%m-%d')
    
     #----------------大盘风控模块，读取风控数据------------------
    risk = 0
    today = data.current_dt.strftime('%Y-%m-%d')
    bm_ret0=ranker_prediction.bm_ret0.values[0]
    bm_ret1=ranker_prediction.bm_ret1.values[0]
    bm_ret2=ranker_prediction.bm_ret2.values[0]
    bm_ret3=ranker_prediction.bm_ret3.values[0]
    bm_risk_v0=ranker_prediction.bm_risk_v0.values[0]
    bm_risk_v1=ranker_prediction.bm_risk_v1.values[0]
    bm_risk_v2=ranker_prediction.bm_risk_v2.values[0]
    
    if bm_ret0 < 0.001:
        if bm_risk_v0 > 0:
            print(today,'大盘放量下跌,全仓卖出')
            risk = 1
        elif bm_ret1 < 0.001 and bm_ret2 < 0.002:
            print(today,'大盘连续下跌,全仓卖出')
            risk = 1
        if bm_ret3 < -0.02:
            print(today,'大盘三日下跌超过2%,全仓卖出')
            risk = 1
    if bm_ret0 > 0.01:
        if (bm_risk_v0 + bm_risk_v1) < 0:
            print(today,'大盘缩量上涨,全仓卖出')
            risk = 1
            
    if risk == 1:
        
        if len(positions)>0:
            # 全部卖出后返回
            for instrument in positions:
                if data.can_trade(context.symbol(instrument)):
                    context.order_target_percent(context.symbol(instrument), 0)
        return # 风控卖出后直接使用return结束当日交易，后续轮仓逻辑不再执行
    #---------------------大盘风控结束--------------------------------------
    
    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    
   #------------------------------------------卖出模块START--------------------------------------------
    if len(positions) > 0:
        for instrument in positions.keys():
            last_sale_date = positions[instrument].last_sale_date   #上次交易日期
            delta_days = data.current_dt - last_sale_date  
            hold_days = delta_days.days #持仓天数
            if hold_days >= 0:
                context.order_target(context.symbol(instrument), 0)
    #-------------------------------------------卖出模块END---------------------------------------------
    
    
    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument

    for i, instrument in enumerate(buy_instruments):
        try:
            cash = cash_for_buy * buy_cash_weights[i]
            if cash > max_cash_per_instrument - positions.get(instrument, 0):
                # 确保股票持仓量不会超过每次股票最大的占用资金量
                cash = max_cash_per_instrument - positions.get(instrument, 0)
            if context.datecont == 1:
                # 获取今天和昨天的成交量
                volume_since_buy = data.history(context.symbol(instrument), 'volume', 3, '1d')
                close_price = data.current(context.symbol(instrument), 'close')  #当收盘价
                high_price = data.current(context.symbol(instrument), 'high')  #当天最高价
                if ((volume_since_buy[2]/volume_since_buy[1] < 2.5) or (high_price/close_price<1.05)) and volume_since_buy[2]/volume_since_buy[0] > 1:
                    current_price = data.current(context.symbol(instrument), 'price')
                    amount = math.floor(cash / current_price - cash / current_price % 100)
                    context.order(context.symbol(instrument), amount)
                    return
                else:
                    print('today = ',today,'instrument = ',instrument)
        except:
            print('today = ',today,'instrument = ',instrument)
# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass

def m12_generate_sql_bigquant_run():
    # 使用sql读取数据
    sql = "select version()"
    return sql
    #return ""


def m21_generate_sql_bigquant_run():
    # 使用sql读取数据
    sql = "select * from wp_options limit 10"
    return sql
    #return ""


m1 = M.instruments.v2(
    start_date='2015-02-01',
    end_date='2019-05-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -2) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
rank_return_10/rank_return_30
avg_turn_15/turn_0
ta_cci_14_0
return_0
turn_5/avg_turn_10
swing_volatility_10_0/swing_volatility_60_0
rank_amount_10/rank_amount_30
list_days_0
mf_net_amount_xl_0
avg_turn_5
turn_0/avg_turn_5
rank_return_20
rank_return_5/rank_return_20
avg_amount_5
rank_avg_turn_5"""
)

m5 = M.input_features.v1(
    features_ds=m3.data,
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
close_0
high_1
open_0
low_0
st_status_0"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m5.data,
    start_date='',
    end_date='',
    before_start_days=60
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m5.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m10 = M.filter.v3(
    input_data=m7.data,
    expr='st_status_0==0 and low_0>high_1 and close_0>open_0',
    output_left_data=False
)

m13 = M.dropnan.v1(
    input_data=m10.data
)

m4 = M.stock_ranker_train.v6(
    training_ds=m13.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2019-10-23'),
    end_date=T.live_run_param('trading_date', '2020-03-09'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m5.data,
    start_date='',
    end_date='',
    before_start_days=60
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m5.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m11 = M.filter.v3(
    input_data=m18.data,
    expr='st_status_0==0 and low_0>high_1+0.02 and close_0>open_0',
    output_left_data=False
)

m14 = M.dropnan.v1(
    input_data=m11.data
)

m8 = M.stock_ranker_predict.v5(
    model=m4.model,
    data=m14.data,
    m_lazy_run=False
)

m20 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
ret_1=close/shift(close,1)
ret_3=close/shift(close,3)
volumepct_1=volume/shift(volume,1)
bm_ret0=ret_1
bm_ret1=shift(bm_ret0,1)
bm_ret2=shift(bm_ret0,2)
bm_ret3=ret_3
bm_risk_v0=volumepct_1
bm_risk_v1=shift(bm_risk_v0,1)
bm_risk_v2=shift(bm_risk_v0,2)"""
)

m6 = M.index_feature_extract.v3(
    input_1=m9.data,
    input_2=m20.data,
    before_days=100,
    index='000001.HIX'
)

m25 = M.join.v3(
    data1=m8.predictions,
    data2=m6.data_1,
    on='date',
    how='left',
    sort=False
)

m26 = M.sort.v4(
    input_ds=m25.data,
    sort_by='date,position',
    group_by='--',
    keep_columns='--',
    ascending=True
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m26.sorted_data,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)

m12 = M.datahub_load_postgres.v1(
    conn='jyquant:Cong2021@DB009@121.41.121.43:5433',
    database='jyquant_db',
    generate_sql=m12_generate_sql_bigquant_run
)

m21 = M.datahub_load_db.v1(
    conn='www_blenderget_:z8Db2XXZNc@121.41.121.43',
    database='www_blenderget_',
    generate_sql=m21_generate_sql_bigquant_run,
    charset='utf8'
)


[2021-11-08 11:54:50.776352] INFO: 读取数据(Postgres): 读取数据

###  <font color=black>读取数据(Postgres) <font color='DimGray' size=3> 数据统计 <font color='gray' size=3> (前 1 行)</font> </font></font>

,version
count(Nan),0
type,object


###  <font color=black>读取数据(Postgres) <font color='DimGray' size=3> 数据预览 <font color='gray' size=3> (前 5 行)</font> </font></font>

,version
0,"PostgreSQL 13.0 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44), 64-bit"


[2021-11-08 11:54:50.793619] INFO: moduleinvoker: datahub_load_postgres.v1 运行完成[0.88991s].

In [2]:
m6.data_1.read()

,date,bm_ret0,bm_ret1,bm_ret2,bm_ret3,bm_risk_v0,bm_risk_v1,bm_risk_v2
0,2019-07-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-07-16,0.998447,NaN,NaN,NaN,0.786263,NaN,NaN
2,2019-07-17,0.997983,0.998447,NaN,NaN,1.135192,0.786263,NaN
3,2019-07-18,0.989591,0.997983,0.998447,0.986062,1.001719,1.135192,0.786263
4,2019-07-19,1.007936,0.989591,0.997983,0.995433,0.997203,1.001719,1.135192
5,2019-07-22,0.987270,1.007936,0.989591,0.984747,1.208774,0.997203,1.001719
6,2019-07-23,1.004493,0.987270,1.007936,0.999575,0.715708,1.208774,0.997203
7,2019-07-24,1.008046,1.004493,0.987270,0.999684,1.200566,0.715708,1.208774
8,2019-07-25,1.004817,1.008046,1.004493,1.017453,0.938411,1.200566,0.715708
9,2019-07-26,1.002445,1.004817,1.008046,1.015378,0.956474,0.938411,1.200566
